# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd

clients = pd.read_csv('/Users/grigorij/Desktop/Портфолио/Исследование надежности заемщиков/data.csv') # чтение файла с данными и сохранение в clients

clients.head(10) # получение первых 10 строк таблицы clients

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
clients.info()# получение общей информации о данных в таблице clients

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
print(clients.isna().sum())# подсчёт пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


**Наблюдения:**

1) Как видим, данные о стаже хранятся в днях, однако многие из них - отрицательные. А также имеются строки, которых данные исчисляются сотнями лет, что физически невозможно. Ниже посмотрим, как много таких значений и от чего они зависят. 

2) В столбцах со стажем работы и ежемесячным доходом имеется 2174 пропуска. Так как это практически 10% всех данных, их необходимо обработать. В разделе с обработкой пропусков затем проверим, в каких именно ячейках они содержатся и соотносятся ли друг с другом (т.к. есть предположение, что это одни и те же строки).

3) Столбцы `days_employed` и `total_income` имеют формат с плавающей точкой, однако для наших расчетов достаточно целочисленного значения. 

Чтобы получить первичное представление о статистических характеристиках нашего датасета, используем метод describe:

In [4]:
clients.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Наблюдения:** 

1) "Подозрительные" данные в столбце с количеством детей: -1 и 20 (с отрицательным значением - возможно техническая ошибка; с большим количеством - нужно проверять, была это ошибка ввода, или уникальный случай). Проверим, сколько таких строк, а также другие возможные аномальные значения.

2) Снова подтверждается наблюдение о том, что данные о стаже заполнены некорректно - минимальное значение отрицательное, а максимальное значение составляет 1100 лет. Перед заполнением пропусков, нужно будет обработать и привести эти данные в порядок.

3) Минимальный возраст клиента - 0 лет, нужно проверить, сколько таких строк, и либо избавиться от них, либо заменить на подходящее значение. Также можно проверить, сколько клиентов с возрастом меньше 18 лет.

### Шаг 2.1 Заполнение пропусков

Сначала проверим, пропущенные значения стобцов находятся в одних и тех же строках, или нет:


In [5]:
clients[(clients['days_employed'].isnull() == True) & (clients['total_income'].isnull() == True)].info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Пропущенные значения находятся в одних и тех же строках. Узнаем, в каких группах состоят эти клиенты:

In [6]:
clients[(clients['days_employed'].isnull() == True) & (clients['total_income'].isnull() == True)]['income_type'].value_counts()


сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Значит, для подстановки вместо нулевых значений, имеет смысл использовать усредненные значения в разбивке по группам. В данном случае лучше использовать медиану, т.к. разброс данных относительно среднего значения значительный.

Мы уже можем подставить данные в столбец с заработком `total_income`, однако необходимо привести в порядок столбец со стажем работы, а также исправить другие аномалии. 

### Шаг 2.2 Проверка данных на аномалии и исправления.

Посмотрим, сколько отрицательных и положительных значений в столбце `days_employed`, и попробуем сделать вывод, как их можно исправить для дальнейшей работы.

In [7]:
print("Строк в 'days_employed' > 0:", clients[clients['days_employed'] > 0]['days_employed'].shape[0])
print("Строк в 'days_employed' < 0:", clients[clients['days_employed'] < 0].shape[0])

#выведем количество клиентов, чей стаж больше максимального - 47 лет (65 лет минус 18 лет)

clients[(clients['days_employed'].abs() > 17155)]['income_type'].value_counts() #сколько людей имеют стаж больше максимально возможного 47 лет, в разбивке по категориям


Строк в 'days_employed' > 0: 3445
Строк в 'days_employed' < 0: 15906


пенсионер      3443
безработный       2
сотрудник         1
компаньон         1
Name: income_type, dtype: int64

In [8]:
clients['income_type'].value_counts() #выведем для информации количество клиентов по категориям

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Большинство строк, выходящих за максимальное значение, относятся к категории "пенсионер" и "безработный". Скорее всего это может быть связано с особенностями обработки анкет этих категорий клиентов (не предоставляют документы о своей работе, а опираются на другие документы, например).

Предположим, что данные по стажу в этих категориях можно ограничить по максимально возможной планке в 47 лет. 

Две других строчки из категории "сотрудник" и "компаньон" - можно заменить медианным значением по категорииб либо так же заменить на максимально возможный.

Отрицательные значения предлагаю взять по модулю, предположив, что это ошибка в отображении информации (а в остальном - стаж в рамках реальных цифр).

In [9]:
clients['days_employed'] = clients['days_employed'].abs() #сделаем данные о стаже положительными
clients.loc[(clients['days_employed'] > 17155),'days_employed'] = 17155 #заменим данные, выходящие за пределы максимального на максимальное
clients[(clients['days_employed'] == 17155)]['income_type'].value_counts() #выведем количество замененных строк для проверки


пенсионер      3443
безработный       2
сотрудник         1
компаньон         1
Name: income_type, dtype: int64

Исправим ошибки в столбце `children`:

In [10]:
print("Количество строк с 'children' -1 -", clients[clients['children'] == -1].count()[0])
print("Количество строк с 'children' 20 -", clients[clients['children'] == 20].count()[0])

Количество строк с 'children' -1 - 47
Количество строк с 'children' 20 - 76


In [11]:
#заменим количество детей -1 на 0
clients['children'] = clients['children'].replace(-1, 0)
print("Количество строк с 'children' -1 -", clients[clients['children'] == -1].count()[0])

Количество строк с 'children' -1 - 0


Таким образом, избавились от строчек с отрицательным количеством детей. Данные в строках с количеством 20 детей не единичные, поэтому оставим их и потом будем учитывать как многодетных (потому что не знаем, чем именно вызвана ошибка данных). В идеале уточнить эту информацию у источника данных.

Теперь заменим данные в строках с возрастом меньше 18 лет на медианные по группе:

In [12]:
print("Количество строк с 'dob_years' 0 -", clients[clients['dob_years'] == 0].count()[0])
print("Количество строк с 'dob_years' < 18 -", clients[clients['dob_years'] < 18].count()[0])

Количество строк с 'dob_years' 0 - 101
Количество строк с 'dob_years' < 18 - 101


In [13]:
#все значения меньше 18 - нулевые 
#заменим данные в этих строках на медианные 

clients.loc[(clients['dob_years'] < 18),'dob_years'] = clients.groupby('income_type')['dob_years'].transform('median')
print("Количество строк с 'dob_years' 0 -", clients[clients['dob_years'] == 0].count()[0]) #проверим факт замены

Количество строк с 'dob_years' 0 - 0


In [14]:
#проверим столбец gender
clients['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [15]:
#удалим строку с неопределенным полом
clients = clients[clients['gender'] != 'XNA']
clients['gender'].value_counts() #проверим удаление

F    14236
M     7288
Name: gender, dtype: int64

Завершив все преобразования, финально заполним все пропуски в столбцах `days_employed` и `total_income` медианными значениями в разбивке по группам:

In [16]:
clients['days_employed'] = clients['days_employed'].fillna(clients.groupby('income_type')['days_employed'].transform('median'))
clients['total_income'] = clients['total_income'].fillna(clients.groupby('income_type')['total_income'].transform('median'))
clients.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,17155.000000,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50.0,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35.0,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [17]:
print(clients.isna().sum())# проверим замену пропусков


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Шаг 2.3. Изменение типов данных.

In [18]:
clients.info() #проверим типы данных в датасете

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21524 non-null  int64  
 1   days_employed     21524 non-null  float64
 2   dob_years         21524 non-null  float64
 3   education         21524 non-null  object 
 4   education_id      21524 non-null  int64  
 5   family_status     21524 non-null  object 
 6   family_status_id  21524 non-null  int64  
 7   gender            21524 non-null  object 
 8   income_type       21524 non-null  object 
 9   debt              21524 non-null  int64  
 10  total_income      21524 non-null  float64
 11  purpose           21524 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.1+ MB


In [19]:
#заменим тип данных с плавающей точкой на целочисленный, так как это не мешает нашим исследованиям, а представление в таблице будет более читабельным
clients['dob_years'] = clients['dob_years'].astype('int')
clients['days_employed'] = clients['days_employed'].astype('int')
clients['total_income'] = clients['total_income'].astype('int')
clients.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21524 non-null  int64 
 1   days_employed     21524 non-null  int64 
 2   dob_years         21524 non-null  int64 
 3   education         21524 non-null  object
 4   education_id      21524 non-null  int64 
 5   family_status     21524 non-null  object
 6   family_status_id  21524 non-null  int64 
 7   gender            21524 non-null  object
 8   income_type       21524 non-null  object
 9   debt              21524 non-null  int64 
 10  total_income      21524 non-null  int64 
 11  purpose           21524 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


In [20]:
display(clients.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,17155,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Шаг 2.4. Удаление дубликатов.

Проверим названия в столбцах `education`

In [21]:
print(clients['education'].sort_values().unique())# Проверка на неявные дубликаты

['ВЫСШЕЕ' 'Высшее' 'НАЧАЛЬНОЕ' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Начальное'
 'Неоконченное высшее' 'СРЕДНЕЕ' 'Среднее' 'УЧЕНАЯ СТЕПЕНЬ'
 'Ученая степень' 'высшее' 'начальное' 'неоконченное высшее' 'среднее'
 'ученая степень']


In [22]:
#приведем их к единому виду с помощью low

clients['education'] = clients['education'].str.lower()
print(clients['education'].sort_values().unique()) #и проверим

['высшее' 'начальное' 'неоконченное высшее' 'среднее' 'ученая степень']


In [23]:
clients.duplicated().sum() #посмотрим количество дубликатов

71

In [24]:
clients[clients.duplicated(keep=False)].sort_values(by=['dob_years', 'purpose']) #выведем их на экран и проверим

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15892,0,1574,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,142594,сделка с подержанным автомобилем
19321,0,1574,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,142594,сделка с подержанным автомобилем
8853,1,1574,23,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу
20297,1,1574,23,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу
3452,0,1574,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594,покупка жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
5865,0,17155,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью
9528,0,17155,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью
6537,0,17155,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
7938,0,17155,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы


In [25]:
#дубликаты выглядят как задвоенные, значит их можно удалить без вреда для исследования
clients = clients.drop_duplicates() #удаляем
clients.duplicated().sum() #проверяем наличие

0

Предположим, что дубликаты могли возникнуть вследствие технической ошибки (например, заявка на подтверждение кредита отправлялась дважды и записались обе строки).

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма, в которых:

- каждому уникальному значению из `education` соответствует уникальное значение `education_id` — сохраним в `education_dict`;
- каждому уникальному значению из `family_status` соответствует уникальное значение `family_status_id` — сохраним в `family_dict`.

In [26]:
education_dict=clients[['education_id','education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
display(education_dict.sort_values(by = 'education_id'))

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [27]:
family_dict=clients[['family_status_id','family_status']]
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
display(family_dict.sort_values(by = 'family_status_id'))

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


<div class="alert alert-success"; style="border-left: 7px solid green">
<h5> ✔️ <font color="green">Комментарий ревьюера (зеленый)</font></h5>

Правильно созданы новые таблицы!

</div>

In [28]:
#по ТЗ для проекта необходимо удалить столбцы education и family_status из датасета, 
#однако это затруднит дальнейшую работу с ним, поэтому я просто приведу код для их удаления здесь, но без его обработки:

#clients.drop(['family_status', 'education'], axis=1, inplace=True)
#display(clients.head(10))


### Шаг 2.6. Категоризация дохода.

In [29]:
def total_income_category(total_income): #напишем функцию для присвоения категории в зависимости от величины среднемесячного дохода
    if total_income <= 30000:
        return 'E'
    if total_income >=30001 and total_income <= 50000:
        return 'D'
    if total_income >=50001 and total_income <= 200000:
        return 'C'
    if total_income >=200001 and total_income <= 1000000:
        return 'B'
    if total_income >=1000001:
        return 'A'


clients['total_income_category'] = clients['total_income'].apply(total_income_category) #добавим новый столбец в датасет с этими данными
display(clients.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,17155,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.

In [30]:
print(clients['purpose'].sort_values().unique()) #выберем вручную основы слов, и на их основе напишем функцию

['автомобили' 'автомобиль' 'высшее образование'
 'дополнительное образование' 'жилье' 'заняться высшим образованием'
 'заняться образованием' 'на покупку автомобиля'
 'на покупку подержанного автомобиля' 'на покупку своего автомобиля'
 'на проведение свадьбы' 'недвижимость' 'образование' 'операции с жильем'
 'операции с коммерческой недвижимостью' 'операции с недвижимостью'
 'операции со своей недвижимостью' 'покупка жилой недвижимости'
 'покупка жилья' 'покупка жилья для сдачи' 'покупка жилья для семьи'
 'покупка коммерческой недвижимости' 'покупка недвижимости'
 'покупка своего жилья' 'получение высшего образования'
 'получение дополнительного образования' 'получение образования'
 'приобретение автомобиля' 'профильное образование' 'ремонт жилью'
 'свадьба' 'свой автомобиль' 'сделка с автомобилем'
 'сделка с подержанным автомобилем' 'строительство жилой недвижимости'
 'строительство недвижимости' 'строительство собственной недвижимости'
 'сыграть свадьбу']


In [31]:
def purpose_correction(row):
    try:
        if 'недвижим' in row['purpose'] or 'жил' in row['purpose']:
            return 'операции с недвижимостью'
        elif 'автомобил' in row['purpose']:
            return 'операции с автомобилем'
        elif 'образован' in row['purpose']:
            return 'получение образования'
        elif 'свадьб' in row['purpose']:
            return 'проведение свадьбы'
    except:
            return 'ошибка обработки'
    
clients['purpose'] = clients.apply(purpose_correction,axis = 1) #присвоим каждой строке одну из 4 целей
display(clients.head(10)) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,операции с недвижимостью,B
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,операции с автомобилем,C
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,операции с недвижимостью,C
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,получение образования,B
4,0,17155,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,проведение свадьбы,C
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,операции с недвижимостью,B
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с недвижимостью,B
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,получение образования,C
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,проведение свадьбы,C
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,операции с недвижимостью,C


### Ответы на вопросы.

#### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок? 

In [32]:
def children_category(row): #функция присваивает категорию в зависимости от количества детей
    if row['children'] == 0:
        return 'нет детей'
    elif 1 <= row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'
    
clients['children_category'] = clients.apply(children_category, axis=1) #добавим временный столбец с данными о категории

#создадим сводную таблицу и укажем в ней процент просроченных платежей для каждой из категории

clients_pivot = clients.pivot_table(index='children_category', columns= 'debt', values='days_employed', aggfunc='count')
clients_pivot['percent'] = round(clients_pivot[1] / (clients_pivot[0] + clients_pivot[1]), 3)*100
display(clients_pivot.sort_values('percent', ascending = False))

clients.drop(['children_category'], axis=1, inplace=True) #удалим временный столбец


debt,0,1,percent
children_category,,,
1-2 ребенка,6222,638,9.3
многодетные,417,39,8.6
нет детей,13073,1064,7.5


**Вывод:**

Заемщики, имеющие детей склонны задерживать платеж по кредиту, в отличие от бездетных. Скорее всего это связано с тем, что у семей с детьми больше расходов, и например траты на детей будут стоять в приоритете перед тратами на погашение кредита.

Стоит отметить, выборка по многодетным не такая большая, как по остальным группам. 

#### Вопрос 2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [33]:
#создадим сводную таблицу и укажем в ней процент просроченных платежей для каждой из категории

clients_pivot = clients.pivot_table(index='family_status', columns= 'debt', values='days_employed', aggfunc='count')
clients_pivot['percent'] = round(clients_pivot[1] / (clients_pivot[0] + clients_pivot[1]), 3)*100
clients_pivot.sort_values('percent', ascending = False)

debt,0,1,percent
family_status,,,
Не женат / не замужем,2536,274,9.8
гражданский брак,3762,388,9.3
женат / замужем,11408,931,7.5
в разводе,1110,85,7.1
вдовец / вдова,896,63,6.6


**Вывод:**

Заемщики, не состоящие в официальных отношениях, почти на 2% чаще пропускают платежи, чем люди состоящие в браке или находящиеся в разводе. При этом люди, оставшиеся без супруга, возвращают кредиты в срок чаще всех остальных.

Можно предположить, что человек в браке становится более ответственным, либо совместный бюджет позволяет обслуживать кредит более качественно.

#### Вопрос 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [34]:
#создадим сводную таблицу и укажем в ней процент просроченных платежей для каждой из категории

clients_pivot = clients.pivot_table(index='total_income_category', columns= 'debt', values='days_employed', aggfunc='count')
clients_pivot['percent'] = round(clients_pivot[1] / (clients_pivot[0] + clients_pivot[1]), 3)*100
clients_pivot.sort_values('percent', ascending = True)


debt,0,1,percent
total_income_category,,,
D,329,21,6.0
B,4685,356,7.1
A,23,2,8.0
C,14655,1360,8.5
E,20,2,9.1


**Вывод:**

Реже всего задолженность по кредиту возникает у клиентов с ежемесячным доходом до 50000 рублей, а также у клиентов с доходом выше среднего. Можно предположить, что эти категории лучше распоряжаются своими финансами. Однако явной закономерности между уровнем дохода и ответственным возвратом кредита - нет.

Однако выборка на категориях А, Е и D - не такая широкая, как в категориях В и С, разница между которыми порядка 1,5 процента. Это не очень большая разница, возможно следует вооспользоваться другими критериями, чтобы дать более точный ответ.

#### Вопрос 4. Как разные цели кредита влияют на его возврат в срок?

In [35]:
#создадим сводную таблицу и укажем в ней процент просроченных платежей для каждой из категории

clients_pivot = clients.pivot_table(index='purpose', columns= 'debt', values='days_employed', aggfunc='count')
clients_pivot['percent'] = round(clients_pivot[1] / (clients_pivot[0] + clients_pivot[1]), 3)*100
clients_pivot.sort_values('percent', ascending = True)


debt,0,1,percent
purpose,,,
операции с недвижимостью,10028,782,7.2
проведение свадьбы,2138,186,8.0
получение образования,3643,370,9.2
операции с автомобилем,3903,403,9.4


**Вывод:**

Заемщики, берущие кредит для приобретения жилья, наиболее ответственны и менее склонны нарушать обязательства по выплатам кредита в срок. Возможно это более масштабное событие в жизни человека, и он относится к выплате ипотеки/кредита на недвижимость более ответственно. 

Интересно, что люди, которые оплачивают свадьбу из кредитных денег, имеют меньше просрочек по платежам, чем те, кто берет деньги на машину или учебу. 



### Общий вывод:

Вопрос от заказчика звучит так:  "Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок".

Судя по выводам из предыдущих вопросов, можно сказать:

1) **Семейное положение влияет на своевременное погашение кредита.** Клиенты с официальными текущими или прошлыми отношениями чаще выплачивают кредит в срок, чем одинокие клиенты, или клиентв в гражданском браке. При этом вдовцы - самые ответственные заемщики.

2) Кредит на свадьбу возвращают вовремя чаще, чем займы на автомобиль/образование.

3) Человек, **имеющий детей склонен задерживать выплаты** по кредиту в отличие от бездетных заемщиков.

**Портрет клиента**

Можно предположить, что клиент, берущий ипотеку, при этом состоящий в браке и не имеющий детей окажется самым ответственным заемщиком.

И наоборот, многодетная семья, оформляющая кредит на машину, с большей вероятностью будет возвращать его с задержками.

**Замечание**

Однако, разница в показателях возвратимости кредитов по разным категориям клиентов составляет в среднем 1-2%. Имеет смысл проводить более глубокий анализ, чтобы сделать более детальные выводы. Например, при более детальной разбивке по целям кредита, мужчины склонны чаще задерживать выплаты по обязательствам, чем женщины:


In [36]:
clients_pivot = clients.pivot_table(index=['purpose','gender'], columns= 'debt', values='days_employed', aggfunc='count')
clients_pivot['percent'] = round(clients_pivot[1] / (clients_pivot[0] + clients_pivot[1]), 3)*100
clients_pivot.sort_values('purpose', ascending = True)


debt                                0    1  percent
purpose                  gender                    
операции с автомобилем   F       2625  235      8.2
                         M       1278  168     11.6
операции с недвижимостью F       6641  444      6.3
                         M       3387  338      9.1
получение образования    F       2460  207      7.8
                         M       1183  163     12.1
проведение свадьбы       F       1454  108      6.9
                         M        684   78     10.2

В данном случае, я бы обсудил результаты исследования с заказчиком, и в соответствии с его пожеланиями, рассмотрел результаты в различных разрезах.